In [1]:
import cycpd
import matplotlib.pyplot as plt
# import pycpd
import numpy as np
import os
import glob
import shutil
import open3d as o3d
import time
import h5py
import pandas as pd
from scipy.spatial.transform import Rotation as RR
import ssm_utils
import projection_utils as proj

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Gaussian Process & Multivariate Normal Distribution

In [2]:
paramsFiles = glob.glob(os.path.join(r"./data/params", "sRtParams_*.hdf5"))
toothIndices = [int(os.path.basename(f).split(".")[0].split("_")[-1]) for f in paramsFiles]

In [3]:
dfParam = pd.DataFrame(columns=["tag"])
for f,tID in zip(paramsFiles,toothIndices):
    tags, scales, rotMats, transVecs = ssm_utils.readRegistrationParamsFromHDF5(h5File=f, toothIndex=tID)
    rotAngles = ssm_utils.getRotAngles(rotMats)
    rx, ry, rz = np.hsplit(rotAngles, 3)
    tx, ty, tz = np.hsplit(transVecs, 3)
    tags = [tag[:-1] for tag in tags]
    dataTemp = {"tag":tags, 
                "{}s".format(tID):scales, 
                "{}rx".format(tID):rx.flatten(), 
                "{}ry".format(tID):ry.flatten(), 
                "{}rz".format(tID):rz.flatten(),
                "{}tx".format(tID):tx.flatten(), 
                "{}ty".format(tID):ty.flatten(), 
                "{}tz".format(tID):tz.flatten()} 
    dfTemp = pd.DataFrame(dataTemp)
    dfParam = dfParam.merge(dfTemp, how="outer", on="tag")

In [37]:
dfParam

,tag,11s,11rx,11ry,11rz,11tx,11ty,11tz,12s,12rx,...,46tx,46ty,46tz,47s,47rx,47ry,47rz,47tx,47ty,47tz
0,65,1.575657,-0.417926,-0.481351,0.249453,10.117639,-2.097857,0.277476,1.218601,-0.328558,...,1.302736,-6.269171,-1.038112,1.040174,0.015166,0.143515,-0.531078,-0.859560,-13.322393,-5.262710
1,87,1.221864,0.217351,-0.037992,-0.276693,2.923883,2.645940,-4.395098,1.026329,0.047310,...,-1.715662,-2.563255,2.736868,0.936835,0.069393,-0.156241,-0.084532,-5.121869,-2.473417,2.940782
2,84,1.243813,-0.291333,0.464359,-0.150202,-10.057017,-3.592691,-1.126615,1.034752,-0.298091,...,-2.753845,-8.467804,-0.974047,0.942282,0.179305,0.160814,-0.623633,-5.942495,-16.496163,-6.611356
3,4,1.167750,0.069135,-0.140152,-0.037460,3.247426,1.080696,-3.003360,1.189040,0.043493,...,1.429975,1.252429,-1.776484,1.135607,0.011708,0.021838,0.012897,2.474693,0.062724,1.736226
4,111,1.171392,-0.324573,-0.445354,-0.031206,6.563996,-5.149365,3.251975,1.132319,-0.206623,...,5.772693,1.391698,-1.303359,1.251403,-0.112454,-0.020584,-0.066664,7.595420,0.337184,6.086091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,72,0.869813,-0.021190,0.176258,-0.150886,-2.404517,-0.221881,1.556239,0.888826,-0.053049,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.097345,0.105345,...,0.884411,-1.768931,-1.862404,1.017284,-0.012574,-0.054611,-0.080661,-0.512782,-1.626821,1.140145
127,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.953832,0.108155,...,-0.459578,0.453675,-1.427462,1.021983,-0.056423,-0.056087,0.381929,-5.032038,9.070757,3.986438
128,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.877819,-0.056448,...,-0.539552,-1.343462,0.074190,0.964833,-0.033392,0.154054,-0.086390,1.969761,-0.782816,-4.397893


In [5]:
DATA_DIR = r"D:/Smartee/"
SSM_DIR = os.path.join(DATA_DIR, r"data/cpdGpAlignedData/eigValVec/")
CPD_ALIGNED_DIR = os.path.join(DATA_DIR, r"data/cpdGpAlignedData/")
PARAM_DIR = os.path.join(DATA_DIR, r"data/cpdGpParams/") 
SRC_DIR = os.path.join(DATA_DIR, r"dataWithPhoto/cpdGpAlignedData/")
SRC_PARAM_DIR = os.path.join(DATA_DIR, r"dataWithPhoto/cpdGpParams/")
PG_NPY = os.path.join(SRC_PARAM_DIR, "Y_pg.npy")
MASK_NPY = os.path.join(SRC_PARAM_DIR, "X_mask.npy")
NAME_IDX_MAP_CSV = os.path.join(DATA_DIR, r"dataWithPhoto/nameIndexMapping.csv")
NAME_IDX_CSV = pd.read_csv(NAME_IDX_MAP_CSV)
NUM_PC = 10
NUM_POINT = 1500
PG_SHAPE = (NUM_POINT, 3)
UPPER_INDICES = [11,12,13,14,15,16,17,21,22,23,24,25,26,27] #不考虑智齿18,28
LOWER_INDICES = [31,32,33,34,35,36,37,41,42,43,44,45,46,47] #不考虑智齿38,48
_TeethIds = np.array(UPPER_INDICES + LOWER_INDICES)

In [6]:
def updateAbsTransVecs(invParamDF, Mu):
    """将牙列transVecShift融入txyz中,忽略牙列scale的影响,并且将txyz定义在局部坐标系下
    使得initPG = inv_s * (meanPG + invTransVec) @ invR + transVecShift
    变为initPG = inv_s * (meanPG - MuCentroids) @ invR + AbsTransVec + MuCentroids"""
    toothIndices = UPPER_INDICES+LOWER_INDICES
    invParamCopy = invParamDF.copy()
    numSample = invParamDF.shape[0]
    numTooth = len(toothIndices)
    X_Mu_centroids = {tID:Mu[i].mean(axis=0) for i,tID in enumerate(toothIndices)}
    invScalesColumns = ["{}s".format(id) for id in toothIndices]
    invRotVecXYZColumns = ["{}r{}".format(id, p) for id in toothIndices for p in ["x","y","z"]]
    invTransVecXYZColumns = ["{}t{}".format(id, p) for id in toothIndices for p in ["x","y","z"]]
    invTransVecShiftColumns = ["upper_ts", "lower_ts"]
    invScales = invParamDF[invScalesColumns].to_numpy()
    invRotVecs = invParamDF[invRotVecXYZColumns].to_numpy().reshape(numSample, numTooth, 3)
    invTransVecs = invParamDF[invTransVecXYZColumns].to_numpy().reshape(numSample, numTooth, 3)
    invTransVecShifts = np.concatenate([np.stack(invParamDF["upper_ts"].to_list()), np.stack(invParamDF["lower_ts"].to_list())], axis=1)
    
    for i in range(numSample):
        for j,tID in enumerate(toothIndices):
            rxyz = invRotVecs[i,j]
            invRotMat = RR.from_rotvec(rxyz).as_matrix().T
            # invTx, invTy, invTz = - X_Mu_centroids[tID] + invScales[i,j] * (invTransVecs[i,j] + X_Mu_centroids[tID] - invTransVecShifts[i,j]) @ invRotMat # Wrong
            invTx, invTy, invTz = - X_Mu_centroids[tID] + invTransVecShifts[i,j] + invScales[i,j] * (invTransVecs[i,j] + X_Mu_centroids[tID]) @ invRotMat # Correct
            invParamCopy.loc[i,"{}tx".format(tID)] = invTx
            invParamCopy.loc[i,"{}ty".format(tID)] = invTy
            invParamCopy.loc[i,"{}tz".format(tID)] = invTz
    invParamCopy = invParamCopy.drop(labels=["upper_s", "lower_s", "upper_ts", "lower_ts"], axis=1)
    return invParamCopy

In [7]:

Mu, SqrtEigVals, Sigma = proj.loadMuEigValSigma(SSM_DIR, numPC=NUM_PC)
invRegistrationParamDF = proj.loadInvRegistrationParams(loadDir=PARAM_DIR) # 加载配准过程中的参数
invParamDF = updateAbsTransVecs(invRegistrationParamDF, Mu) # 将牙列scale转化为每颗牙齿的位移，将每颗牙齿的transVecXYZs在局部坐标系下进行表达

In [8]:
tc = np.mean(Mu, axis=1)

In [41]:
dfParam = dfParam.astype(np.float32)

In [42]:
# 验证txyz的计算是否正确
errors = []
for testToothID in UPPER_INDICES:
    j = np.argmin(_TeethIds<testToothID)
    for testSampleID in range(50):
        testFile = r"D:\Smartee\data\cpdGpAlignedData\{}\{}U.txt".format(testToothID, testSampleID)
        if os.path.exists(testFile):
            _s = invParamDF[invParamDF['tag']==testSampleID][f"{testToothID}s"].to_numpy()[0]
            _rxyz = invParamDF[invParamDF['tag']==testSampleID][[f"{testToothID}rx",f"{testToothID}ry",f"{testToothID}rz"]].to_numpy()[0]
            _R = RR.from_rotvec(_rxyz).as_matrix().T
            _txyz = invParamDF[invParamDF['tag']==testSampleID][[f"{testToothID}tx",f"{testToothID}ty",f"{testToothID}tz"]].to_numpy()[0]
            
            _s1 = dfParam[dfParam['tag']==testSampleID][f"{testToothID}s"].to_numpy()[0]
            _rxyz1 = dfParam[dfParam['tag']==testSampleID][[f"{testToothID}rx",f"{testToothID}ry",f"{testToothID}rz"]].to_numpy()[0]
            _R1 = RR.from_rotvec(_rxyz1).as_matrix().T
            _txyz1 = dfParam[dfParam['tag']==testSampleID][[f"{testToothID}tx",f"{testToothID}ty",f"{testToothID}tz"]].to_numpy()[0]

            X_row_aligned_init = np.loadtxt(testFile)
            X0 = _s1 * X_row_aligned_init @ _R1 + _txyz1
            X_init = X_row_aligned_init + + invRegistrationParamDF[invRegistrationParamDF['tag']==testSampleID]["upper_ts"].to_numpy()[0][j]
            X_recons = _s * (X0 - tc[j]) @ _R + _txyz + tc[j]
            _e = np.linalg.norm(X_init - X_recons, axis=1).mean()
            if not np.isnan(_e):
                errors.append(_e)

In [43]:
print(np.mean(errors))

0.3451643972316142


In [69]:
tc_flat = tc.flatten()

In [9]:
A = invParamDF[["{}t{}".format(_id, p) for _id in UPPER_INDICES for p in ["x","y","z"]]]
# A = invParamDF[["{}r{}".format(_id, p) for _id in LOWER_INDICES for p in ["x","y","z"]]]
# A = invParamDF[["{}r{}".format(_id, p) for _id in UPPER_INDICES+LOWER_INDICES for p in ["x","y","z"]]]
# A = A + tc_flat[None,:42]
A.shape

(127, 42)

In [10]:
# A = dfParam.iloc[:,1:8].to_numpy()
covA = np.ma.cov(np.ma.masked_invalid(A), rowvar=False)
assert not covA.mask.any() #检查是否有nan
covMat = covA.data
covMat.shape

(42, 42)

In [11]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [12]:
is_pos_def(covMat)

False

In [19]:
infoRatio = 0.95
_eigVals, eigVecs = np.linalg.eig(covMat)
eigVals = np.abs(_eigVals)
cum_info_ratio = np.cumsum(eigVals) / np.sum(eigVals)
numPc = np.argmax(cum_info_ratio > infoRatio) + 1

In [20]:
numPc

14

In [21]:
_numPC = numPc
ssm_utils.remainedInfoRatio(eigVals, _numPC)

0.9529348959978046

In [22]:
np.allclose(eigVecs @ np.diag(_eigVals) @ eigVecs.T, covMat)

True

In [23]:
# 用95%的主成分近似covMat
approxCovMat = eigVecs[:,:_numPC] @ np.diag(eigVals[:_numPC]) @ eigVecs[:,:_numPC].T
err = np.linalg.norm(approxCovMat-covMat, "fro")
print(err)

0.8757705158647301


In [25]:
approxCovMat.shape

(42, 42)

In [24]:
is_pos_def(approxCovMat)

False

In [ ]:
# # subV @ featureVec = observedVec - observedPriorVecMu 最小二乘解featureVec; observedPriorVecMu = 0 for txyz
# V = eigVecs[:,:_numPC]
# subV = V[obserIndices]
# subVc = V[nonObserIndices] 
# lambd = 0.0 # a small constant to avoid over-fitting while maintaining acceptable residuals
# featureVec = np.linalg.inv(subV.T @ subV + lambd*np.identity(_numPC)) @ subV.T @ (observedVec - observedPriorVecMu)

In [153]:
# pd.DataFrame(covMat).to_csv("covMat.csv",index=False,header=False)

In [7]:
variances = np.ma.var(np.ma.masked_invalid(A), axis=0, ddof=1)
assert not variances.mask.any() #检查是否有nan
std = np.sqrt(variances.data)

In [8]:
rhoCoef = covMat / np.multiply(std[:,None],std)
assert (np.abs(rhoCoef.diagonal()-1.)<1e-6).all() #对角线元素全是1

In [9]:
largeRhoCoef = rhoCoef.copy()
largeRhoCoef[np.abs(largeRhoCoef)<0.6] = 0.
pd.DataFrame(largeRhoCoef).round(2).to_csv("rhoCoef.csv",index=False,header=False)